In [172]:
import os
%pwd
import sys, os
sys.path.append(os.path.abspath(".."))  # go one directory up (from notebooks to project)
print(sys.executable)   # must point to kidney2
print(sys.path)         
sys.path.append("/home/yousef/Kidney-Diseases-Classification/src")

/home/yousef/miniconda3/envs/kidney2/bin/python
['/home/yousef/miniconda3/envs/kidney2/lib/python310.zip', '/home/yousef/miniconda3/envs/kidney2/lib/python3.10', '/home/yousef/miniconda3/envs/kidney2/lib/python3.10/lib-dynload', '', '/home/yousef/.local/lib/python3.10/site-packages', '/home/yousef/miniconda3/envs/kidney2/lib/python3.10/site-packages', '/home/yousef/Kidney-Diseases-Classification', '/home/yousef/Kidney-Diseases-Classification', '/home/yousef/Kidney-Diseases-Classification/src', '..', '..', '/home/yousef/Kidney-Diseases-Classification/src', '/home/yousef', '/home/yousef/Kidney-Diseases-Classification/src', '..', '..', '/home', '/home/yousef/Kidney-Diseases-Classification/src', '..', '/home/yousef/Kidney-Diseases-Classification/src', '/', '/home/yousef/Kidney-Diseases-Classification/src', '..', '/home/yousef/Kidney-Diseases-Classification/src', '/', '/home/yousef/Kidney-Diseases-Classification/src', '..', '/home/yousef/Kidney-Diseases-Classification/src', '/', '/home/yous

In [173]:
sys.path.append("..")  # Add parent directory to Python path
import cnnClassifier
print(cnnClassifier.__file__)

/home/yousef/Kidney-Diseases-Classification/src/cnnClassifier/__init__.py


In [174]:
import os
os.chdir("/home/yousef/Kidney-Diseases-Classification")
print("📂 Now working in:", os.getcwd())

📂 Now working in: /home/yousef/Kidney-Diseases-Classification


In [175]:
import os, glob, cnnClassifier

print("cnnClassifier location:", cnnClassifier.__file__)
print("Submodules:", os.listdir(os.path.dirname(cnnClassifier.__file__)))

# Show inside entity/
entity_path = os.path.join(os.path.dirname(cnnClassifier.__file__), "entity")
print("Entity path exists?", os.path.exists(entity_path))
print("Entity contents:", glob.glob(os.path.join(entity_path, "*")))

cnnClassifier location: /home/yousef/Kidney-Diseases-Classification/src/cnnClassifier/__init__.py
Submodules: ['pipeline', '__init__.py', 'constants', 'utils', 'components', 'config', 'entity', '__pycache__']
Entity path exists? True
Entity contents: ['/home/yousef/Kidney-Diseases-Classification/src/cnnClassifier/entity/__init__.py', '/home/yousef/Kidney-Diseases-Classification/src/cnnClassifier/entity/config_entity.py', '/home/yousef/Kidney-Diseases-Classification/src/cnnClassifier/entity/__pycache__']


In [176]:
import os

for file in ["configs/config.yaml", "params.yaml"]:
    full_path = f"/home/yousef/Kidney-Diseases-Classification/{file}"
    print(file, os.path.getsize(full_path))

configs/config.yaml 276
params.yaml 164


In [177]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: str
    unzip_dir: str

In [178]:
# Install the package (explicit path!)
%pip install -e /home/yousef/Kidney-Diseases-Classification

# Ensure src/ is on sys.path
import sys
import zipfile
import gdown
sys.path.append("/home/yousef/Kidney-Diseases-Classification/src")

# Debug: confirm Python can see cnnClassifier
import importlib.util, os
print("cnnClassifier path:", importlib.util.find_spec("cnnClassifier").origin)

# Now imports
from cnnClassifier.utils.common import get_size
from cnnClassifier.entity.config_entity import DataIngestionConfig

Obtaining file:///home/yousef/Kidney-Diseases-Classification
  Preparing metadata (setup.py) ... done
  Attempting uninstall: cnnClassifier
    Found existing installation: cnnClassifier 0.0.1
    Uninstalling cnnClassifier-0.0.1:
      Successfully uninstalled cnnClassifier-0.0.1
  DEPRECATION: Legacy editable install of cnnClassifier==0.0.1 from file:///home/yousef/Kidney-Diseases-Classification (setup.py develop) is deprecated. pip 25.3 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for cnnClassifier
Note: you may need to restart the kernel to use updated packages.
cnnClassifier path: /home/yousef/Kidney-Diseases-Classification/sr

In [ ]:
from pathlib import Path
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.entity.config_entity import DataIngestionConfig
from cnnClassifier import logger

class ConfigManager:
    def __init__(self, 
                 config_filename: str = "configs/config.yaml", 
                 params_filename: str = "params.yaml"):
        
        # Force project root explicitly
        self.project_root = Path("/home/yousef/Kidney-Diseases-Classification")

        # Build absolute paths
        self.config_path = self.project_root / config_filename
        self.params_path = self.project_root / params_filename

        print(f"📂 Looking for config at: {self.config_path}")
        print(f"📂 Looking for params at: {self.params_path}")

        # Validate existence
        if not self.config_path.exists():
            raise FileNotFoundError(f"❌ Config file not found: {self.config_path}")
        if not self.params_path.exists():
            raise FileNotFoundError(f"❌ Params file not found: {self.params_path}")

        # Load YAMLs
        self.config = read_yaml(str(self.config_path))
        self.params = read_yaml(str(self.params_path))

        # Create artifacts directory
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        return DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir),
        )

In [180]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) -> str:
        '''
        fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_path = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading file from :[{dataset_url}] into :[{zip_download_path}]")

            file_id = dataset_url.split('/')[-2]
            prefix = dataset_url.split('/')[-1].split('?')[0]
            gdown.download(id=file_id, output=str(zip_download_path), quiet=False)

            logger.info(f"File :[{zip_download_path}] has been downloaded successfully.")

        except Exception as e:
            raise e
        
    def extract_zip_file(self) -> None:
        '''
        unzip the downloaded file
        '''
        try:
            unzip_path = self.config.unzip_dir
            zip_file_path = self.config.local_data_file

            logger.info(f"Unzipping file :[{zip_file_path}] into dir :[{unzip_path}]")
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Unzipping completed successfully. and data is available at :[{unzip_path}]")
        except Exception as e:
            raise e    
        


In [181]:
try:
    config= ConfigManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config= data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

📂 Looking for config at: /home/yousef/Kidney-Diseases-Classification/configs/config.yaml
📂 Looking for params at: /home/yousef/Kidney-Diseases-Classification/params.yaml
[2025-09-08 19:56:13,816: INFO: common: yaml file: /home/yousef/Kidney-Diseases-Classification/configs/config.yaml loaded successfully]
[2025-09-08 19:56:13,817: INFO: common: yaml file: /home/yousef/Kidney-Diseases-Classification/params.yaml loaded successfully]
[2025-09-08 19:56:13,818: INFO: common: created directory at: ./artifacts]
[2025-09-08 19:56:13,818: INFO: common: created directory at: ./artifacts/data_ingestion]
[2025-09-08 19:56:13,819: INFO: 2738064876: Downloading file from :[https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing] into :[artifacts/data_ingestion/data.zip]]


Downloading...
From (original): https://drive.google.com/uc?id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=61fba301-8130-4bbe-a2da-8ce498270d56
To: /home/yousef/Kidney-Diseases-Classification/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:17<00:00, 3.22MB/s]

[2025-09-08 19:56:34,317: INFO: 2738064876: File :[artifacts/data_ingestion/data.zip] has been downloaded successfully.]
[2025-09-08 19:56:34,319: INFO: 2738064876: Unzipping file :[artifacts/data_ingestion/data.zip] into dir :[artifacts/data_ingestion]]


[2025-09-08 19:56:34,688: INFO: 2738064876: Unzipping completed successfully. and data is available at :[artifacts/data_ingestion]]
